# Introduction #
With the rapid growth of demand for taxi services, accurate fare prediction is becoming an important aspect for optimizing business processes and increasing customer satisfaction. This study aims to develop a model that can predict taxi prices based on various factors such as geographical coordinates (longitude and latitude), pick-up and drop-off times, and time parameters including season, day of the week, and time of day.

Using modern machine learning methods, in particular the LinearRegression algorithm, Random Forest Regressor, XGBoost, I aimed to achieve high accuracy in predictions. The study analyzed various models and their performance in the context of predicting taxi prices. The results showed an impressive accuracy of 99.2% (XGBoost) and 99.7% (Random Forest Regressor), which highlights the potential of using machine learning algorithms to solve problems in the field of transportation services.

This study not only demonstrates the possibilities of forecasting taxi prices, but also opens up new horizons for the implementation of analytical solutions in the business strategies of companies operating in this field.

## Importing the librarires ##

In [ ]:
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xg 
from geopy.distance import geodesic
from sklearn.datasets import make_regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_squared_error as MSE 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
### Loading Uber fare data from a CSV file
data = pd.read_csv("/Users/riteshkumar/Downloads/ML projects/Uber taxi trip cost prediction xgboost/Uber - order history.csv")

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
### Removing a column with unnecessary information
data = data.drop(["Unnamed: 0"],axis=1)
data.head()

In [ ]:
### Defining the target variable "fare_amount" 
target = data["fare_amount"]
features = data.drop(["fare_amount"],axis = 1)
target.head()
features.head()

In [ ]:
data.info()

In [ ]:
### Check null values
data.isnull().sum()

In [ ]:
### Removing null values
data.dropna(inplace = True)

In [ ]:
### Check removing
data.isnull().sum()

In [ ]:
data.shape

In [ ]:
### Check duplicated in data
data.duplicated().sum()

In [ ]:
data.info()

In [ ]:
### Let's limit latitude and longitude to real values
data = data[(data["pickup_latitude"].between(-90,90)) &
           (data["dropoff_latitude"].between(-90,90)) &
           (data["pickup_longitude"].between(-180,180))&
           (data["dropoff_longitude"].between(-180,180))]

In [ ]:
### Extracting year, month, day of week and hour from data
data["pickup_datetime"] = pd.to_datetime(data["pickup_datetime"])

data["year"] = data["pickup_datetime"].dt.year
data["month"] = data["pickup_datetime"].dt.month
data["weekday"] = data["pickup_datetime"].dt.weekday
data["hour"] = data["pickup_datetime"].dt.hour

In [ ]:
### Checking the month number for correctness
data["month"].unique()

In [ ]:
### Combining months into quarters
data["Monthly_Quarter"] = data["month"].apply(lambda x: "Q1" if x in [1,2,3] else(
                                              "Q2" if x in [4,5,6] else( "Q3" if x in [7,8,9] else( 
                                              "Q4"))))

In [ ]:
data["Monthly_Quarter"].unique()

In [ ]:
### Checking the hours number for correctness
data["hour"].unique()

In [ ]:
### Combining hour into quarters segments
data["Hourly_Segments"] = data["hour"].apply(lambda x: "H1" if x in [0,1,2,3] 
                                             else("H2" if x in [4,5,6,7] else("H3" if x in [8,9,10,11]
                                                                             else("H4" if x in [12,13,14,15]
                                                                                 else("H5" if x in [16,17,18,19]
                                                                                     else("H6"))))))

In [ ]:
### Calculate the distance between passenger pick-up and drop-off points (distances)
distances = []

for i in data.index:
    pickup_coords = (data["pickup_latitude"][i], data["pickup_longitude"][i])
    dropoff_coords = (data["dropoff_latitude"][i], data["dropoff_longitude"][i])
    distance = round(geodesic(pickup_coords,dropoff_coords).m,2)
    distances.append(distance)
    
data["distance"] = distances

In [ ]:
data.head()

In [ ]:
### Removing a column with unnecessary information
data.drop(["pickup_datetime","month","hour","key"], axis = 1, inplace = True)

In [ ]:
data.head()

In [ ]:
display(data.describe())

In [ ]:
### Removing data with negative "fare_amount"
data = data[data["fare_amount"] >= 0]

In [ ]:
display(data.describe())

## Exploratory Data Analysis (EDA) ##

In [ ]:
warnings.filterwarnings('ignore')

plt.figure(figsize=[6,4])
sns.histplot(data["fare_amount"], color='b', edgecolor="black", linewidth=2, bins=20, kde=True)

plt.title('Target Variable Distribution - Median Value of Homes ($1Ms)')
plt.show()


In [ ]:
categorical_column = data[['Monthly_Quarter', 'passenger_count', 'Hourly_Segments', 'weekday', 'year']]
categorical_column.head()

In [ ]:
### Let's plot the "passenger_count" distribution in the data
sns.countplot(x=data["passenger_count"], palette="magma")
plt.show()

In [ ]:
sns.kdeplot(data["distance"], shade=True, color="purple")
plt.title("Density Plot of 'Distance'")
plt.xlabel("Distance")
plt.ylabel("Density")
plt.show()



In [ ]:
sns.kdeplot(data["pickup_latitude"], shade=True, color="purple")
plt.title("Density Plot 'pickup_latitude'")
plt.xlabel("pickup_latitude")
plt.ylabel("Density")
plt.show()

In [ ]:
sns.kdeplot(data["dropoff_latitude"], shade=True, color="purple")
plt.title("Density Plot 'dropoff_latitude'")
plt.xlabel("dropoff_latitude")
plt.ylabel("Density")
plt.show()

In [ ]:
sns.kdeplot(data["pickup_longitude"], shade=True, color="purple")
plt.title("Density Plot 'pickup_longitude'")
plt.xlabel("pickup_longitude")
plt.ylabel("Density")
plt.show()

In [ ]:
sns.kdeplot(data["dropoff_longitude"], shade=True, color="purple")
plt.title("Density Plot 'dropoff_longitude'")
plt.xlabel("dropoff_longitude")
plt.ylabel("Density")
plt.show()

In [ ]:
sns.kdeplot(data["fare_amount"], shade=True, color="purple")
plt.title("Density Plot 'fare_amount'")
plt.xlabel("dropoff_longitude")
plt.ylabel("Density")
plt.show()

## Data Preprocessing ##

In [ ]:
df = data.copy(deep=True)

In [ ]:
df.shape

In [ ]:
data.columns

In [ ]:
### Let's place categorical features in a variable
data_nf = data[["fare_amount","pickup_longitude","pickup_latitude","dropoff_longitude","dropoff_latitude","distance"]]

In [ ]:
data_nf.head()

In [ ]:
### Let's place quantitative features in a variable
data_cf = pd.get_dummies(data,columns = ['Monthly_Quarter', 'Hourly_Segments', 'weekday', 'year', 'passenger_count'],dtype = int, drop_first=True)

In [ ]:
data_cf.shape

In [ ]:
### Determine the skew for each variable
data_cf["fare_amount"].skew()

In [ ]:
data_cf["pickup_longitude"].skew()

In [ ]:
data_cf["pickup_latitude"].skew()

In [ ]:
data_cf["dropoff_longitude"].skew()

In [ ]:
data_cf["dropoff_latitude"].skew()

In [ ]:
data_cf["distance"].skew()

In [ ]:
data_cf["fare_amount"].describe()

In [ ]:
plt.figure(figsize = (8,5))
sns.distplot(data_cf["fare_amount"], color="purple")
plt.show()

In [ ]:
### Calculation of 25th and 75th percentiles
percentile25 = data_cf["distance"].quantile(0.25)
percentile75 = data_cf["distance"].quantile(0.75)

In [ ]:
### Calculation Interquartile Range
IQR = percentile75 - percentile25
IQR

In [ ]:
### Defining limits
upperlimit = percentile75 + 1.5 * IQR
lowerlimit = percentile25 - 1.5 * IQR

In [ ]:
data_cf[data_cf["distance"] > upperlimit]

In [ ]:
data_cf[data_cf["distance"] < lowerlimit]

In [ ]:
data_cf = data_cf.drop(data_cf[data_cf["distance"] > upperlimit].index)
data_cf.shape

In [ ]:
### Check new skew
data_cf["distance"].skew()

In [ ]:
plt.figure(figsize = (10,5))
sns.distplot(data_cf["distance"], color="purple")
plt.show()


In [ ]:
data_cf["pickup_longitude"].skew()

In [ ]:
### Calculation of 25th and 75th percentiles
percentile25 = data_cf["pickup_longitude"].quantile(0.25)
percentile75 = data_cf["pickup_longitude"].quantile(0.75)

In [ ]:
### Calculation Interquartile Range
IQR = percentile75 - percentile25

In [ ]:
### Defining limits
upperlimit = percentile75 + 1.5 * IQR
lowerlimit = percentile25 - 1.5 * IQR

In [ ]:
data_cf[data_cf["pickup_longitude"] > upperlimit]

In [ ]:
data_cf[data_cf["pickup_longitude"] < lowerlimit]

In [ ]:
data_cf = data_cf.drop(data_cf[data_cf["pickup_longitude"] > upperlimit].index)

In [ ]:
data_cf = data_cf.drop(data_cf[data_cf["pickup_longitude"] < lowerlimit].index)

In [ ]:
data_cf.shape

In [ ]:
### Check new skew
data_cf["pickup_longitude"].skew()

In [ ]:
plt.figure(figsize = (10,5))
sns.distplot(data_cf["pickup_longitude"], color="purple")
plt.show()

In [ ]:
### Calculation of 25th and 75th percentiles
percentile25 = data_cf["pickup_latitude"].quantile(0.25)
percentile75 = data_cf["pickup_latitude"].quantile(0.75)

In [ ]:
### Calculation Interquartile Range
IQR = percentile75 - percentile25
IQR

In [ ]:
### Defining limits
upperlimit = percentile75 + 1.5 * IQR
lowerlimit = percentile25 - 1.5 * IQR

In [ ]:
data_cf[data_cf["pickup_latitude"] > upperlimit]

In [ ]:
data_cf[data_cf["pickup_latitude"] < lowerlimit]

In [ ]:
data_cf = data_cf.drop(data_cf[data_cf["pickup_latitude"] > upperlimit].index)
data_cf = data_cf.drop(data_cf[data_cf["pickup_latitude"] < lowerlimit].index)
data_cf.shape

In [ ]:
### Check new skew
data_cf["pickup_latitude"].skew()

In [ ]:
### Calculation of 25th and 75th percentiles
percentile25 = data_cf["dropoff_longitude"].quantile(0.25)
percentile75 = data_cf["dropoff_longitude"].quantile(0.75)

In [ ]:
### Calculation Interquartile Range
IQR = percentile75 - percentile25
IQR

In [ ]:
### Defining limits
upperlimit = percentile75 + 1.5 * IQR
lowerlimt = percentile25 - 1.5 * IQR

In [ ]:
data_cf[data_cf["dropoff_longitude"] > upperlimit]


In [ ]:
data_cf[data_cf["dropoff_longitude"] < lowerlimt]

In [ ]:
data_cf = data_cf.drop(data_cf[data_cf["dropoff_longitude"] > upperlimit].index)
data_cf = data_cf.drop(data_cf[data_cf["dropoff_longitude"] < lowerlimt].index)
data_cf.shape

In [ ]:
### Check new skew
data_cf["dropoff_longitude"].skew()

In [ ]:
### Calculation of 25th and 75th percentiles
percentile25 = data_cf["dropoff_latitude"].quantile(0.25)
percentile75 = data_cf["dropoff_latitude"].quantile(0.75)

In [ ]:
### Calculation Interquartile Range
IQR = percentile75 - percentile25
IQR

In [ ]:
### Defining limits
upperlimit = percentile75 + 1.5 * IQR
lowerlimit = percentile25 - 1.5 * IQR

In [ ]:
data_cf[data_cf["dropoff_latitude"] > upperlimit]


In [ ]:
data_cf[data_cf["dropoff_latitude"] < lowerlimit]

In [ ]:
data_cf = data_cf.drop(data_cf[data_cf["dropoff_latitude"] > upperlimit].index)
data_cf = data_cf.drop(data_cf[data_cf["dropoff_latitude"] < lowerlimit].index)
data_cf.shape

In [ ]:
### Check new skew
data_cf["dropoff_latitude"].skew()

In [ ]:
data_cf.hist(bins=50, figsize=(25, 10), color="purple")
plt.tight_layout()
plt.show()

In [ ]:
data_cf.shape

## Data Manipulation ##

In [ ]:
### Splitting the data into training and test sets
X = data_cf.drop(["fare_amount"],axis = 1)
y = data_cf["fare_amount"]

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=100)

print(X.shape,y.shape,X_train.shape,X_test.shape,y_train.shape,y_test.shape)

In [ ]:
### Sets standardization
std = StandardScaler()

X_train_std = std.fit_transform(X_train)
X_test_std = std.transform(X_test)

In [ ]:
X_train_std

In [ ]:
X_test_std

## LinearRegression ##

In [ ]:
LinearRegression = LinearRegression()

In [ ]:
### Model training
LinearRegression.fit(X_train_std, y_train)

In [ ]:
### Checking the accuracy result of Linear Regression
print("Train Accuracy", LinearRegression.score(X_train_std,y_train))
print("Test Accuracy", LinearRegression.score(X_test_std,y_test))

## Ridge of Regression ##

In [ ]:
ridge = Ridge(random_state=0,alpha=0.1)

In [ ]:
### Model training
ridge.fit(X_train,y_train)

In [ ]:
### Checking the accuracy result of Ridge of Regression
print("Train Accuracy",ridge.score(X_train,y_train))
print("Test Accuracy", ridge.score(X_test,y_test))

## Random Forest Regressor ##

In [ ]:
### Settings and Splitting the data into training and test sets
X,y = make_regression(n_features=3, n_informative=1,random_state=0, shuffle=False)
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=100)

In [ ]:
### Model training
RandomForest = RandomForestRegressor(max_depth=11, random_state=0)
RandomForest.fit(X_train,y_train)

In [ ]:
### Checking the accuracy result of Random Forest Regressor
print("Train Accuracy", RandomForest.score(X_train,y_train))
print("Test Accuracy", RandomForest.score(X_test,y_test))

## XGBoost ##

In [ ]:
### Setting model XGBRegressor
xgb_r = xg.XGBRegressor(objective ='reg:linear', max_depth = 6,
                        max_leaf_nodes = 2,
                        n_estimators = 165, seed = 123) 

In [ ]:
### Model training
xgb_r.fit(X_train,y_train)

In [ ]:
### Predicting values on a test set
y_pred = xgb_r.predict(X_test)
y_pred

In [ ]:
### Calculation of MSE
rmse = np.sqrt(MSE(y_test, y_pred)) 
print("RMSE : % f" %(rmse)) 

In [ ]:
### Checking the accuracy result of XGBRegressor
print("Train Accuracy",xgb_r.score(X_train,y_train))
print("Test Accuracy", xgb_r.score(X_test,y_test))